<a href="https://colab.research.google.com/github/AsutoshDalei/WalkEye/blob/master/MINST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Importing all necessary packages

In [0]:
import numpy as np
import tensorflow as tf 
import tensorflow_datasets as tfds 

Importing our dataset

In [0]:
mdata,minfo=tfds.load(name='mnist',as_supervised=True,with_info=True) #minfo contains information about the dataset
mtrain,mtest=mdata['train'],mdata['test'] #Defining test and train data

We here define the number of validation, train and test samples and convert them into int64 type.

In [0]:
noval=0.2*minfo.splits['train'].num_examples #Number of validation samples
noval=tf.cast(noval,tf.int64)
notrain=minfo.splits['train'].num_examples #Number of train samples
notrain=tf.cast(notrain,tf.int64)
notest=minfo.splits['test'].num_examples #Number of test samples
notest=tf.cast(notest,tf.int64)

Scaling is a important preprocssing procedure. Here we define a function to scale the images from 0-255 to 0-1. We then scale all the images using python's map function

In [0]:
def scale(image,label):
  image=tf.cast(image,tf.float32)
  image/=255.
  return (image,label)
  
scaledtrainval=mtrain.map(scale)
scaledtest=mtest.map(scale)

We shuffle the dataset to have a random variation of data in out dataset. Significance of *buffer_size* is to shuffle data in batches instead of as a whole, to be computationally efficient.

In [0]:
shuffletrainval=scaledtrainval.shuffle(buffer_size=100)
shuffletest=scaledtest.shuffle(buffer_size=100)

Here, we divide the data into valdata for validation and traindata for training. This is done by python functions *take(x)* and *skip(x)*, which take and skip *x* number of samples respectively.

In [0]:
valdata=shuffletrainval.take(noval)
traindata=shuffletrainval.skip(noval)

Here, we divide the data into batches. Train data is divided into batches based on *batchsize*. Since test and validation data need not be batched, they are batched into one single batch of their own size.

In [0]:
batchsize=1000
traindata=traindata.batch(batchsize)
valdata=valdata.batch(noval)
testdata=shuffletest.batch(notest)

In [0]:
valip,valtar=next(iter(valdata)) #Dividing valdation data into sample and target

**Defining a model here**. We first flatten our input images into a one dimensional vector which is then fed into a deep neural network. The activation of last layer should we softmax as we are expecting an output based on probabilities.

In [0]:
ipsize=784
opsize=10
hiddensize=100
model=tf.keras.Sequential([
                           tf.keras.layers.Flatten(input_shape=(28,28,1)),
                           tf.keras.layers.Dense(100,activation='tanh'),
                           tf.keras.layers.Dense(100,activation='sigmoid'),
                           tf.keras.layers.Dense(100,activation='relu'),
                           tf.keras.layers.Dense(opsize,activation='softmax'),
                        ])

**Compiling our model**. We add a custom optimizer so that we can adjust the learning rate. Since we are dealing with a classification problem and have not applied one hot encoding, we shall use sparce categorical crossentropy as our loss function.

In [0]:
custopt=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=custopt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

**Train(fitting) the model to look at the progress.**

In [64]:
model.fit(traindata,validation_data=(valip,valtar),epochs=10,verbose=1)

Epoch 1/10
48/48 [==============================] - 11s 235ms/step - loss: 0.8393 - accuracy: 0.7815 - val_loss: 0.3445 - val_accuracy: 0.8995
Epoch 2/10
48/48 [==============================] - 11s 232ms/step - loss: 0.2845 - accuracy: 0.9156 - val_loss: 0.2393 - val_accuracy: 0.9306
Epoch 3/10
48/48 [==============================] - 12s 243ms/step - loss: 0.2152 - accuracy: 0.9361 - val_loss: 0.1981 - val_accuracy: 0.9415
Epoch 4/10
48/48 [==============================] - 12s 252ms/step - loss: 0.1766 - accuracy: 0.9483 - val_loss: 0.1721 - val_accuracy: 0.9468
Epoch 5/10
48/48 [==============================] - 12s 260ms/step - loss: 0.1492 - accuracy: 0.9566 - val_loss: 0.1537 - val_accuracy: 0.9529
Epoch 6/10
48/48 [==============================] - 12s 242ms/step - loss: 0.1287 - accuracy: 0.9622 - val_loss: 0.1397 - val_accuracy: 0.9570
Epoch 7/10
48/48 [==============================] - 11s 235ms/step - loss: 0.1124 - accuracy: 0.9670 - val_loss: 0.1295 - val_accuracy: 0.9604

**Evaluating our model with test data**

In [78]:
testloss,testaccuracy=model.evaluate(testdata)
print('TestLoss: {0:.4f}. Test Accuracy: {1:.3f}'.format(testloss,testaccuracy*100))

1/1 [==============================] - 0s 2ms/step - loss: 0.1045 - accuracy: 0.9680
TestLoss: 0.1045. Test Accuracy: 96.800
